# Guided generation from a Regex Schema

In [3]:
import outlinesmlx as outlines

model = outlines.models.mlx("TinyLlama/TinyLlama-1.1B-Chat-v0.6")

prompt = "What is the IP address of the Google DNS servers? "

generator = outlines.generate.text(model)
unguided = generator(prompt, max_tokens=30)

generator = outlines.generate.regex(
    model,
    r"((25[0-5]|2[0-4]\d|[01]?\d\d?)\.){3}(25[0-5]|2[0-4]\d|[01]?\d\d?)",
)
guided = generator(prompt, max_tokens=30)

print("Prompt: ",prompt)

print("\n Result, unguided: \n",unguided)
# What is the IP address of the Google DNS servers?
#
# Passive DNS servers are at DNS servers that are private.
# In other words, both IP servers are private. The database
# does not contain Chelsea Manning

print("\n Result, guided: \n",guided)
# What is the IP address of the Google DNS servers?
# 2.2.6.1

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

Prompt:  What is the IP address of the Google DNS servers? 

 Result, unguided: 
 2. What is the country/region where the Google DNS servers are located? 3. According to this page, can I use Google custom domains

 Result, guided: 
 2.1.2.168


# Guided generation from a Pydantic Model

In [10]:
import outlinesmlx as outlines
from enum import Enum
from pydantic import BaseModel, constr


class Weapon(str, Enum):
    sword = "sword"
    axe = "axe"
    mace = "mace"
    spear = "spear"
    bow = "bow"
    crossbow = "crossbow"


class Armor(str, Enum):
    leather = "leather"
    chainmail = "chainmail"
    plate = "plate"


class Character(BaseModel):
    name: constr(max_length=10)
    age: int
    armor: Armor
    weapon: Weapon
    strength: int

model = outlines.models.mlx("TinyLlama/TinyLlama-1.1B-Chat-v0.6")
# Construct guided sequence generator
generator = outlines.generate.json(model, Character, max_tokens=100)

# Draw a sample
character = generator("Give me a character description")

print(repr(character))

character = generator("Give me an interesting character description")

print(repr(character))

Character(name='Alice', age=35, armor=<Armor.leather: 'leather'>, weapon=<Weapon.axe: 'axe'>, strength=90)
Character(name='Sophia', age=25, armor=<Armor.leather: 'leather'>, weapon=<Weapon.sword: 'sword'>, strength=30)


# Guided generation from a JSON schema

In [9]:
import outlinesmlx as outlines

schema = '''{
    "title": "Character",
    "type": "object",
    "properties": {
        "name": {
            "title": "Name",
            "maxLength": 10,
            "type": "string"
        },
        "age": {
            "title": "Age",
            "type": "integer"
        },
        "armor": {"$ref": "#/definitions/Armor"},
        "weapon": {"$ref": "#/definitions/Weapon"},
        "strength": {
            "title": "Strength",
            "type": "integer"
        }
    },
    "required": ["name", "age", "armor", "weapon", "strength"],
    "definitions": {
        "Armor": {
            "title": "Armor",
            "description": "An enumeration.",
            "enum": ["leather", "chainmail", "plate"],
            "type": "string"
        },
        "Weapon": {
            "title": "Weapon",
            "description": "An enumeration.",
            "enum": ["sword", "axe", "mace", "spear", "bow", "crossbow"],
            "type": "string"
        }
    }
}'''

model = outlines.models.mlx("microsoft/phi-2")
generator = outlines.generate.json(model, schema)
character = generator("Give me a character description", max_tokens=2048)
print(character)

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


JSONDecodeError: Expecting value: line 1446 column 5 (char 4705)

# Context free grammars

In [6]:
import outlinesmlx as outlines

arithmetic_grammar = """
    ?start: expression

    ?expression: term (("+" | "-") term)*

    ?term: factor (("*" | "/") factor)*

    ?factor: NUMBER
           | "-" factor
           | "(" expression ")"

    %import common.NUMBER
"""

model = outlines.models.mlx("microsoft/phi-2")
generator = outlines.generate.cfg(model, arithmetic_grammar)
sequence = generator("Alice had 4 apples and Bob ate 2. Write an expression for Alice's apples:",max_tokens=300)

print(sequence)

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


4-2.5-2.3-1.5-1.7-2.7-1.9-2.1-1.6-0.9-2.3-1.7-1.3-2.0-2.0-2.1-2.1-0.0-2.8-2.8-3.1-4.2-2.3-2.6-1.9-1.4-1.9-1.5-1.7-1.5-2.2-1.8-1.1-0.9-2.6-2.8-3.0-1.6-0.3-2.1-1.7-1.4-1.8-2.0-2.2-2.2-0.0-2.9-2.4-3.3-2.7-1.0-2.6-0.3-1.1-2.0-1.9-2.5-0.8-1.9-2.0-0.1-2.1-2.0-2.1-2.4+2.7+3.3-4.2+1.3+0.6-0.4-0.0-0.1-0.2-2.


# Open Functions

In [4]:
import outlinesmlx as outlines


def add(a: int, b: int):
    return a + b

model = outlines.models.mlx("TinyLlama/TinyLlama-1.1B-Chat-v0.6")
generator = outlines.generate.json(model, add)
result = generator("Return json with two integers named a and b respectively. a is odd and b even.")

print("result:",result)
print(add(**result))


Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

result: {'a': 8, 'b': 4}
12
